# Graph Coloring

The **"Graph Coloring"** quantum kata is a series of exercises designed
to teach you the basics of using Grover search to solve constraint
satisfaction problems, using graph coloring problem as an example.

* [This Microsoft Learn module](https://docs.microsoft.com/learn/modules/solve-graph-coloring-problems-grovers-search/) walks you through solving graph coloring problems using Grover's search.
* You can read more about graph coloring problems [here](https://en.wikipedia.org/wiki/Graph_coloring).
* It is strongly recommended to complete the [Grover's Algorithm kata](./../GroversAlgorithm/GroversAlgorithm.ipynb) before proceeding to this one. You can also refer to its [README.md](./../GroversAlgorithm/README.md) for the list of resources on Grover's algorithm.
* [SolveSATWithGrover](./../SolveSATWithGrover/SolveSATWithGrover.ipynb) is another kata covering oracle implementation for solving constraint satisfaction problems.


Each task is wrapped in one operation preceded by the description of the task.
Your goal is to fill in the blank (marked with the `// ...` comments)
with some Q# code that solves the task. To verify your answer, run the cell using Ctrl+Enter (⌘+Enter on macOS).

Within each section, tasks are given in approximate order of increasing difficulty; 
harder ones are marked with asterisks.

## Part I. Colors Representation and Manipulation

### Task 1.1. Initialize register to a color

**Inputs:** 

  1. An integer $C$ ($0 \leq C \leq 2^{N} - 1$).

  2. An array of $N$ qubits in the $|0...0\rangle$ state.

**Goal:** 

Prepare the array in the basis state which represents the binary notation of $C$. 
Use little-endian encoding (i.e., the least significant bit should be stored in the first qubit).

**Example:** For $N = 2$ and $C = 2$ the state should be $|01\rangle$.

In [1]:
%kata T11_InitializeColor 

operation InitializeColor (C : Int, register : Qubit[]) : Unit is Adj {
    let N = Length(register);
    for i in 0..N-1 {
        if (C / 2^i % 2 == 1) {
            X(register[i]);
        }
    }
}

Success!

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-1.1.-Initialize-register-to-a-color).*

### Task 1.2. Read color from a register

**Input:** An array of $N$ qubits which are guaranteed to be in one of the $2^{N}$ basis states.

**Output:** 

An $N$-bit integer that represents this basis state, in little-endian encoding. 
The operation should not change the state of the qubits.

**Example:** For $N = 2$ and the qubits in the state $|01\rangle$ return 2 (and keep the qubits in $|01\rangle$).

In [2]:
%kata T12_MeasureColor 

open Microsoft.Quantum.Arrays;

operation MeasureColor (register : Qubit[]) : Int {
    mutable color = 0;
    for i in IndexRange(register) {
        let qValue = M(register[i]) == Zero ? 0 | 1;
        set color += qValue * 2^i;
    }
    return color;
}

Success!

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-1.2.-Read-color-from-a-register).*

### Task 1.3. Read coloring from a register

**Inputs:** 

  1. The number of elements in the coloring $K$.

  2. An array of $K * N$ qubits which are guaranteed to be in one of the $2^{KN}$ basis states.

**Output:** 

An array of $K$ $N$-bit integers that represent this basis state. 
$i$-th integer of the array is stored in qubits with indices $i * N$, $i * N + 1$, ..., $i * N + N - 1$ in little-endian format. 
The operation should not change the state of the qubits.

**Example:** 
For $N = 2$, $K = 2$ and the qubits in the state $|0110\rangle$ return `[2, 1]`.

In [3]:
%kata T13_MeasureColoring 

operation MeasureColoring (K : Int, register : Qubit[]) : Int[] {
    mutable colors = new Int[0];
    let N = Length(register)/K;
    
    for i in 0..K-1 {
        set colors += [MeasureColor(register[i*N..(i+1)*N-1])];
    }
    
    return colors;
}

/snippet_.qs(4,22): warning QS3308: Deprecated syntax. Use [] to construct an empty array, or [x, size = n] to construct an array of x repeated n times.


Success!

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-1.3.-Read-coloring-from-a-register).*

### Task 1.4. 2-bit color equality oracle

**Inputs:** 

  1. An array of 2 qubits in an arbitrary state $|c_{0}\rangle$ representing the first color.

  2. An array of 2 qubits in an arbitrary state $|c_{1}\rangle$ representing the second color.

  3. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:**

Transform state $|c_{0}\rangle|c_{1}\rangle|y\rangle$ into state $|c_{0}\rangle|c_{1}\rangle|y \oplus f(c_{0},c_{1})\rangle$ ($\oplus$ is addition modulo 2), 
where $f(x) = 1$ if $c_{0}$ and $c_{1}$ are in the same state, and 0 otherwise. 
Leave the query register in the same state it started in.

In this task you are allowed to allocate extra qubits.

In [4]:
%kata T14_ColorEqualityOracle_2bit 

operation ColorEqualityOracle_2bit (c0 : Qubit[], c1 : Qubit[], target : Qubit) : Unit is Adj+Ctl {
    let N = Length(c0);
    use q = Qubit[N];
    within {
        for i in 0..N-1 {
            CNOT(c0[i], q[i]);
            CNOT(c1[i], q[i]);
        }
    } apply {
        ControlledOnInt(0, X)(q, target);
    }
}

Success!

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-1.4.-2-bit-color-equality-oracle).*

### Task 1.5. N-bit color equality oracle (no extra qubits)

This task is the same as task 1.4, but in this task you are NOT allowed to allocate extra qubits.

In [2]:
%kata T15_ColorEqualityOracle_Nbit 

operation ColorEqualityOracle_Nbit (c0 : Qubit[], c1 : Qubit[], target : Qubit) : Unit is Adj+Ctl {
    let N = Length(c0);
    within {
        for i in 0..N-1 {
            CNOT(c0[i], c1[i]);
        }
    } apply {
        ControlledOnInt(0, X)(c1, target);
    }
}

Success!

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-1.5.-N-bit-color-equality-oracle-(no-extra-qubits)).*

## Part II. Vertex coloring problem
The vertex graph coloring is a coloring of graph vertices which 
labels each vertex with one of the given colors so that 
no two vertices of the same color are connected by an edge.

### Task 2.1. Classical verification of vertex coloring

**Inputs:** 

  1. The number of vertices in the graph $V$ ($V \leq 6$).

  2. An array of $E$ tuples of integers, representing the edges of the graph ($E \leq 12$).  
Each tuple gives the indices of the start and the end vertices of the edge.  
The vertices are indexed $0$ through $V - 1$.

  3. An array of $V$ integers, representing the vertex coloring of the graph. 
$i$-th element of the array is the color of the vertex number $i$.

**Output:** 

True if the given vertex coloring is valid (i.e., no pair of vertices connected by an edge have the same color), and false otherwise.

**Example:** 

Graph 0 -- 1 -- 2 would have $V = 3$ and `edges = [(0, 1), (1, 2)]`.  
Some of the valid colorings for it would be `[0, 1, 0]` and `[-1, 5, 18]`.

In [6]:
%kata T21_IsVertexColoringValid 

function IsVertexColoringValid (V : Int, edges: (Int, Int)[], colors: Int[]) : Bool {
    mutable isColoringValid = true;
    
    for (v1, v2) in edges {
        if isColoringValid and colors[v1] == colors[v2] {
            set isColoringValid = false;
        }
    }
    
    return isColoringValid;
}

Success!

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-2.1.-Classical-verification-of-vertex-coloring).*

### Task 2.2. Oracle for verifying vertex coloring

**Inputs:**

  1. The number of vertices in the graph $V$ ($V \leq 6$).

  2. An array of $E$ tuples of integers, representing the edges of the graph (E $\leq$ 12).  
Each tuple gives the indices of the start and the end vertices of the edge.  
The vertices are indexed $0$ through $V - 1$.

  3. An array of $2V$ qubits `colorsRegister` that encodes the color assignments.

  4. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:**

Transform state $|x, y\rangle$ into state $|x, y \oplus f(x)\rangle$  ($\oplus$ is addition modulo 2), 
where $f(x) = 1$ if the given vertex coloring is valid, and 0 otherwise. 
Leave the query register in the same state it started in.

Each color in `colorsRegister` is represented as a 2-bit integer in little-endian format. 
See task 1.3 for a more detailed description of color assignments.

In [7]:
%kata T22_VertexColoringOracle 

operation VertexColoringOracle (V : Int, 
                                edges : (Int, Int)[], 
                                colorsRegister : Qubit[], 
                                target : Qubit) : Unit is Adj+Ctl {
    let N = Length(edges);
    let colorSize = 2;
    
    use q = Qubit[N];
    within {
        for i in IndexRange(edges) {
            let (v1, v2) = edges[i];
            ColorEqualityOracle_Nbit(colorsRegister[colorSize*v1..colorSize*(v1+1)-1], 
                                     colorsRegister[colorSize*v2..colorSize*(v2+1)-1], 
                                     q[i]);
        }
    } apply {
        ControlledOnInt(0, X)(q, target);
    }
        
    
}

Testing V = 3, edges = []
Testing V = 4, edges = [(0, 1),(0, 2),(0, 3),(1, 2),(1, 3),(2, 3)]
Testing V = 5, edges = [(4, 0),(2, 1),(3, 1),(3, 2)]
Testing V = 5, edges = [(0, 1),(1, 2),(1, 3),(3, 2),(4, 2),(3, 4)]
Testing V = 5, edges = [(0, 1),(0, 2),(0, 4),(1, 2),(1, 3),(2, 3),(2, 4),(3, 4)]


Success!

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-2.2.-Oracle-for-verifying-vertex-coloring).*

### Task 2.3. Using Grover's search to find vertex coloring

**Inputs:** 

  1. The number of vertices in the graph $V$ ($V \leq 6$).

  2. A marking oracle which implements vertex coloring verification, as implemented in task 2.2.

**Output:** 

A valid vertex coloring for the graph, in a format used in task 2.1.

In [8]:
%kata T23_GroversAlgorithm 

open Microsoft.Quantum.Math;
open Microsoft.Quantum.Random;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Convert;

operation GroversAlgorithm (V : Int, oracle : ((Qubit[], Qubit) => Unit is Adj)) : Int[] {
    let hmColors = Ceiling(Lg((V < 4) ? IntAsDouble(V) | 4.0));
    
    let N = 2*V;
    // Alternative 
    // let N = hmColors*V;

    let iterUpperBound = Truncate(PI() / 4.0 * IntAsDouble(N)) + 1;
    mutable result = [];
    mutable iterCounter = 0;
    
    use (x, y) = (Qubit[N], Qubit());    
    use testQubit = Qubit();
    
    X(y);
    H(y);
    

    repeat {
        set iterCounter += 1;
        Message($"Iteration {iterCounter}");
        let hmIterations = DrawRandomInt(1, iterUpperBound);
        ApplyToEach(H, x);
        
        for i in 1..hmIterations {
            oracle(x, y);
            
            ApplyToEach(H, x);
            
            
            X(x[N-1]);
            ControlledOnInt(0, Z)(x[0..N-2], x[N-1]);
            X(x[N-1]);
            
            ApplyToEach(H, x);
        }
        
        set result = ResultArrayAsBoolArray(MultiM(x));
        oracle(x, testQubit);
    } until M(testQubit) == One
    fixup {
        ResetAll(x + [testQubit]);
    }
    
    mutable ans = [];
    for i in 0..V-1 {
        set ans += [BoolArrayAsInt(result[hmColors*i..hmColors*(i+1)-1])];
    }
    
    ResetAll(x + [y, testQubit]);
    
    return ans;
}

Running on graph V = 3, edges = []
Iteration 1
Got correct coloring [0,3,0]
Running on graph V = 4, edges = [(0, 1),(0, 2),(0, 3),(1, 2),(1, 3),(2, 3)]
Iteration 1
Got correct coloring [3,1,2,0]
Running on graph V = 5, edges = [(4, 0),(2, 1),(3, 1),(3, 2)]
Iteration 1
Got correct coloring [2,2,3,0,3]
Running on graph V = 5, edges = [(0, 1),(1, 2),(1, 3),(3, 2),(4, 2),(3, 4)]
Iteration 1
Iteration 2
Got correct coloring [2,3,2,1,0]
Running on graph V = 5, edges = [(0, 1),(0, 2),(0, 4),(1, 2),(1, 3),(2, 3),(2, 4),(3, 4)]
Iteration 1
Iteration 2
Got correct coloring [3,2,0,3,2]
Running on graph V = 6, edges = [(0, 1),(0, 2),(0, 4),(0, 5),(1, 2),(1, 3),(1, 5),(2, 3),(2, 4),(3, 4),(3, 5),(4, 5)]
Iteration 1
Iteration 2
Iteration 3
Got correct coloring [2,0,1,3,0,1]


Success!

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-2.3.-Using-Grover's-search-to-find-vertex-coloring).*

## Part III. Weak coloring problem
Weak graph coloring is a coloring of graph vertices which 
labels each vertex with one of the given colors so that 
each vertex is either isolated or is connected by an edge
to at least one neighbor of a different color.

### Task 3.1. Determine if an edge contains the vertex

**Inputs:**

   1. An edge denoted by a tuple of integers.
      Each tuple gives the indices of the start and the end vertices of the edge.
   2. An integer denoting the vertex of the graph.

**Output:** 

True if the edge contains the vertex provided, and false otherwise.

**Examples:** 
- edge $(0,1)$ contains vertex $0$, so return true;
- edge $(0,1)$ contains vertex $1$, so return true;
- edge $(2,3)$ does not contain vertex $1$, so return false.

In [4]:
%kata T31_DoesEdgeContainVertex

function DoesEdgeContainVertex (edge: (Int, Int), vertex : Int) : Bool {
    let (v1, v2) = edge;
    return v1 == vertex or v2 == vertex;
}

Success!

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-3.1.-Determine-if-an-edge-contains-the-vertex).*

### Task 3.2. Determine if a vertex is weakly colored
**Inputs:** 

  1. The number of vertices in the graph $V$ ($V \leq 6$).

  2. An array of $E$ tuples of integers, representing the edges of the graph ($E \leq 12$).  
Each tuple gives the indices of the start and the end vertices of the edge.  
The vertices are indexed $0$ through $V - 1$.

  3. An array of $V$ integers, representing the vertex coloring of the graph. 
$i$-th element of the array is the color of the vertex number $i$.

  4. A vertex in the graph, indexed $0$ through $V - 1$.

**Output:**

True if the vertex is weakly colored (i.e., it is connected to at least one neighboring vertex of different color),
and false otherwise.

**Note:**

An isolated vertex (a vertex without neighbors) is considered to be weakly colored.

**Example:**

For vertex $0$, in a graph containing `edges = [(0, 1), (0, 2), (1, 2)]` and  `colors = [0, 1, 0]`, vertex $0$ is weakly colored, since it has color 0 and is connected to vertex 1 which has color 1.

In [10]:
%kata T32_IsVertexWeaklyColored

function IsVertexWeaklyColored (V : Int, edges: (Int, Int)[], colors: Int[], vertex : Int) : Bool {
    mutable isWeak = false;
    mutable isIsolated = true;
    
    for edge in edges {
        if not isWeak and DoesEdgeContainVertex(edge, vertex) {
            set isIsolated = false;
            let (v1, v2) = edge;
            
            if (colors[v1] != colors[v2]) {
                set isWeak = true;
            }
        }
    }
    return isWeak or isIsolated;
}

Success!

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-3.2.-Determine-if-a-vertex-is-weakly-colored).*

### Task 3.3. Classical verification of weak coloring

**Inputs:** 

  1. The number of vertices in the graph $V$ ($V \leq 6$).

  2. An array of $E$ tuples of integers, representing the edges of the graph ($E \leq 12$).  
Each tuple gives the indices of the start and the end vertices of the edge.  
The vertices are indexed $0$ through $V - 1$.

  3. An array of $V$ integers, representing the vertex coloring of the graph. 
$i$-th element of the array is the color of the vertex number $i$.

**Output:** 

True if the given weak coloring is valid (i.e., every vertex is isolated or is connected to at least one neighboring vertex of different color), and false otherwise.

**Example:** 

Consider a graph with $V = 3$ and `edges = [(0, 1), (0, 2), (1, 2)]`.  
Some of the valid colorings for it would be `[0, 1, 0]` and `[-1, 5, 18]`.

In [11]:
%kata T33_IsWeakColoringValid

open Microsoft.Quantum.Bitwise;

function IsWeakColoringValid (V : Int, edges: (Int, Int)[], colors: Int[]) : Bool {
    // It's sad that they do not motivate people to write an optimized classical code
    mutable isVertexWeak = [false, size = V];
    mutable isVertexIsolated = [true, size = V];
    
    for (v1, v2) in edges {
        set isVertexIsolated w/= v1 <- false;
        set isVertexIsolated w/= v2 <- false;
        
        if (colors[v1] != colors[v2]) {
            set isVertexWeak w/= v1 <- true;
            set isVertexWeak w/= v2 <- true;
        }
    }
    
    let a = BoolArrayAsInt(isVertexWeak);
    let b = BoolArrayAsInt(isVertexIsolated);
    
    return Or(a, b) == 2^V-1;
    //mutable isWeak = true;
    //
    //for i in 0..V-1 {
    //    if isWeak {
    //        set isWeak = IsVertexWeaklyColored(V, edges, colors, i);
    //    }
    //}
    
    //return isWeak;
}

Success!

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-3.3.-Classical-verification-of-weak-coloring).*

### Task 3.4. Oracle for verifying if a vertex is weakly colored

**Inputs:**

  1. The number of vertices in the graph $V$ ($V \leq 6$).

  2. An array of $E$ tuples of integers, representing the edges of the graph (E $\leq$ 12).  
Each tuple gives the indices of the start and the end vertices of the edge.  
The vertices are indexed $0$ through $V - 1$.

  3. An array of $2V$ qubits `colorsRegister` that encodes the color assignments.

  4. A qubit in an arbitrary state $|y\rangle$ (target qubit).
  
  5. A vertex in the graph, indexed $0$ through $V - 1$.

**Goal:**

Transform state $|x, y\rangle$ into state $|x, y \oplus f(x)\rangle$  ($\oplus$ is addition modulo 2), 
where $f(x) = 1$ if the given weak coloring is valid, and 0 otherwise. 
Leave the query register in the same state it started in.

Each color in `colorsRegister` is represented as a 2-bit integer in little-endian format. 
See $Task\ 1.3$ for a more detailed description of color assignments.

In [6]:
function getUsedQubits(q : Qubit[], edges: (Int, Int)[], vertex : Int, colorLength : Int) : Qubit[] {
    mutable usedQubits = [];
    
    for edge in edges {
        if DoesEdgeContainVertex(edge, vertex) {
            let (v1, v2) = edge;
            let targetVertex = (v1 == vertex) ? v2 | v1;
            set usedQubits += [q[targetVertex]];
        }
    }
    return usedQubits;
}

getUsedQubits

In [7]:
%kata T34_WeaklyColoredVertexOracle

open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Diagnostics;

operation WeaklyColoredVertexOracle(V : Int, 
                                    edges: (Int, Int)[], 
                                    colorsRegister : Qubit[], 
                                    target : Qubit, 
                                    vertex : Int) : Unit is Adj+Ctl {
    
    use q = Qubit[V];
    
    let colorLength = Length(colorsRegister) / V;
    within {
        for edge in edges {
            let (v1, v2) = edge;
            if DoesEdgeContainVertex(edge, vertex) {
                let targetVertex = (v1 == vertex) ? v2 | v1;
                
                ColorEqualityOracle_Nbit(colorsRegister[colorLength*v1..colorLength*(v1+1)-1],
                                         colorsRegister[colorLength*v2..colorLength*(v2+1)-1], 
                                         q[targetVertex]);
            }                
        }
    } apply {
        let usedQubits = getUsedQubits(q, edges, vertex, colorLength);
        
        if Length(usedQubits) > 0 {
            X(target);
        }
        ControlledOnInt(2^Length(usedQubits)-1, X)(usedQubits, target);
    }
}

Testing V = 3, edges = []
Testing V = 3, edges = []
Testing V = 3, edges = []
Testing V = 4, edges = [(0, 1),(0, 2),(0, 3),(1, 2),(1, 3),(2, 3)]
Testing V = 4, edges = [(0, 1),(0, 2),(0, 3),(1, 2),(1, 3),(2, 3)]
Testing V = 4, edges = [(0, 1),(0, 2),(0, 3),(1, 2),(1, 3),(2, 3)]
Testing V = 4, edges = [(0, 1),(0, 2),(0, 3),(1, 2),(1, 3),(2, 3)]
Testing V = 5, edges = [(4, 0),(2, 1),(3, 1),(3, 2)]
Testing V = 5, edges = [(4, 0),(2, 1),(3, 1),(3, 2)]
Testing V = 5, edges = [(4, 0),(2, 1),(3, 1),(3, 2)]
Testing V = 5, edges = [(4, 0),(2, 1),(3, 1),(3, 2)]
Testing V = 5, edges = [(4, 0),(2, 1),(3, 1),(3, 2)]
Testing V = 5, edges = [(0, 1),(1, 2),(1, 3),(3, 2),(4, 2),(3, 4)]
Testing V = 5, edges = [(0, 1),(1, 2),(1, 3),(3, 2),(4, 2),(3, 4)]
Testing V = 5, edges = [(0, 1),(1, 2),(1, 3),(3, 2),(4, 2),(3, 4)]
Testing V = 5, edges = [(0, 1),(1, 2),(1, 3),(3, 2),(4, 2),(3, 4)]
Testing V = 5, edges = [(0, 1),(1, 2),(1, 3),(3, 2),(4, 2),(3, 4)]
Testing V = 5, edges = [(0, 1),(0, 2),(0, 4),(1, 2),(1

Success!

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-3.4.-Oracle-for-verifying-if-a-vertex-is-weakly-colored).*

### Task 3.5. Oracle for verifying weak coloring
    
**Inputs:**

  1. The number of vertices in the graph $V$ ($V \leq 6$).

  2. An array of $E$ tuples of integers, representing the edges of the graph ($E \leq 12$).  
Each tuple gives the indices of the start and the end vertices of the edge.  
The vertices are indexed $0$ through $V - 1$.

  3. An array of $2V$ qubits `colorsRegister` that encodes the color assignments.

  4. A qubit in an arbitrary state $|y\rangle$ (target qubit).

**Goal:**

Transform state $|x, y\rangle$ into state $|x, y \oplus f(x)\rangle$  ($\oplus$ is addition modulo 2), 
where $f(x) = 1$ if the given weak coloring is valid, and 0 otherwise. 
Leave the query register in the same state it started in.

Each color in `colorsRegister` is represented as a 2-bit integer in little-endian format. 
See $Task\ 1.3$ for a more detailed description of color assignments.

In [8]:
%kata T35_WeakColoringOracle

operation WeakColoringOracle (
    V : Int,
    edges : (Int, Int)[],
    colorsRegister : Qubit[],
    target : Qubit
) : Unit is Adj+Ctl {

    use q = Qubit[V];
    
    within {
        for i in 0..V-1 {
            WeaklyColoredVertexOracle(V, edges, colorsRegister, q[i], i);
        }
    }
    apply {
        ControlledOnInt(2^V-1, X)(q, target);
    }    
}

Testing V = 3, edges = []
Testing V = 4, edges = [(0, 1),(0, 2),(0, 3),(1, 2),(1, 3),(2, 3)]
Testing V = 5, edges = [(4, 0),(2, 1),(3, 1),(3, 2)]
Testing V = 5, edges = [(0, 1),(1, 2),(1, 3),(3, 2),(4, 2),(3, 4)]


Success!

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-3.5.-Oracle-for-verifying-weak-coloring).*

### Task 3.6. Using Grover's search to find weak coloring

**Inputs:** 

  1. The number of vertices in the graph $V$ ($V \leq 6$).

  2. A marking oracle which implements weak coloring verification, as implemented in Task 3.5.

**Output:** 

A valid weak coloring for the graph, in a format used in Task 3.3.

In [10]:
%kata T36_GroversAlgorithmForWeakColoring

// This is one to one copypaste from the previous Grover search task...


open Microsoft.Quantum.Math;
open Microsoft.Quantum.Random;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Convert;

operation GroversAlgorithmForWeakColoring (V : Int, oracle : ((Qubit[], Qubit) => Unit is Adj)) : Int[] {
    let hmColors = Ceiling(Lg((V < 4) ? IntAsDouble(V) | 4.0));
    
    let N = 2*V;
    // Alternative 
    // let N = hmColors*V;

    let iterUpperBound = Truncate(PI() / 4.0 * IntAsDouble(N)) + 1;
    mutable result = [];
    mutable iterCounter = 0;
    
    use (x, y) = (Qubit[N], Qubit());    
    use testQubit = Qubit();
    
    X(y);
    H(y);
    

    repeat {
        set iterCounter += 1;
        Message($"Iteration {iterCounter}");
        let hmIterations = DrawRandomInt(1, iterUpperBound);
        ApplyToEach(H, x);
        
        for i in 1..hmIterations {
            oracle(x, y);
            
            ApplyToEach(H, x);
            
            
            X(x[N-1]);
            ControlledOnInt(0, Z)(x[0..N-2], x[N-1]);
            X(x[N-1]);
            
            ApplyToEach(H, x);
        }
        
        set result = ResultArrayAsBoolArray(MultiM(x));
        oracle(x, testQubit);
    } until M(testQubit) == One
    fixup {
        ResetAll(x + [testQubit]);
    }
    
    mutable ans = [];
    for i in 0..V-1 {
        set ans += [BoolArrayAsInt(result[hmColors*i..hmColors*(i+1)-1])];
    }
    
    ResetAll(x + [y, testQubit]);
    
    return ans;
}

Running on graph V = 3, edges = []
Iteration 1
Got correct coloring [0,1,1]
Running on graph V = 4, edges = [(0, 1),(0, 2),(0, 3),(1, 2),(1, 3),(2, 3)]
Iteration 1
Got correct coloring [3,1,1,3]
Running on graph V = 5, edges = [(4, 0),(2, 1),(3, 1),(3, 2)]
Iteration 1
Got correct coloring [0,0,0,2,1]
Running on graph V = 5, edges = [(0, 1),(1, 2),(1, 3),(3, 2),(4, 2),(3, 4)]
Iteration 1
Got correct coloring [3,1,2,3,3]
Running on graph V = 5, edges = [(0, 1),(0, 2),(0, 4),(1, 2),(1, 3),(2, 3),(2, 4),(3, 4)]
Iteration 1
Iteration 2
Got correct coloring [2,1,2,1,0,3]


Success!

*Can't come up with a solution? See the explained solution in the [Graph Coloring Workbook](./Workbook_GraphColoring.ipynb#Task-3.6.-Using-Grover's-search-to-find-weak-coloring).*

## Part IV. Triangle-free coloring problem

Triangle-free graph coloring is a coloring of graph edges which labels each edge with one of two colors so that no three edges of the same color form a triangle.

### Task 4.1. Convert the list of graph edges into an adjacency matrix

**Inputs:**

   1. The number of vertices in the graph $V$ ($V \leq 6$).
   
   2. An array of $E$ tuples of integers, representing the edges of the graph ($E \leq 12$). Each tuple gives the indices of the start and the end vertices of the edge. The vertices are indexed $0$ through $V - 1$.

**Output:**  
A 2D array of integers representing this graph as an adjacency matrix:  the element [i][j] should be $-1$ if the vertices i and j are not connected with an edge, or store the index of the edge if the vertices i and j are connected with an edge. Elements [i][i] should be $-1$ unless there is an edge connecting vertex i to itself.

**Example:**
Consider a graph with $V = 3$ and edges = `[(0, 1), (0, 2), (1, 2)]`.
The adjacency matrix for it would be:

$$
\begin{bmatrix}
-1 &  0 &  1 \\
 0 & -1 &  2 \\
 1 &  2 & -1
\end{bmatrix}
$$


In [16]:
%kata T41_EdgesListAsAdjacencyMatrix

function EdgesListAsAdjacencyMatrix (V : Int, edges : (Int, Int)[]) : Int[][] {
    mutable arr = [[-1, size=V], size=V];
    for i in IndexRange(edges) {
        let (v1, v2) = edges[i];
        set arr w/= v1 <- arr[v1][0..v2-1] + [i] + arr[v1][v2+1..V-1]; 
        set arr w/= v2 <- arr[v2][0..v1-1] + [i] + arr[v2][v1+1..V-1]; 
    }
    return arr;
}

Running on graph V = 3, edges = []
Got correct adjacency matrix
Running on graph V = 4, edges = [(0, 1),(0, 2),(0, 3),(1, 2),(1, 3),(2, 3)]
Got correct adjacency matrix
Running on graph V = 5, edges = [(4, 0),(2, 1),(3, 1),(3, 2)]
Got correct adjacency matrix
Running on graph V = 5, edges = [(0, 1),(1, 2),(1, 3),(3, 2),(4, 2),(3, 4)]
Got correct adjacency matrix
Running on graph V = 5, edges = [(0, 1),(0, 2),(0, 4),(1, 2),(1, 3),(2, 3),(2, 4),(3, 4)]
Got correct adjacency matrix
Running on graph V = 6, edges = [(0, 1),(0, 2),(0, 4),(0, 5),(1, 2),(1, 3),(1, 5),(2, 3),(2, 4),(3, 4),(3, 5),(4, 5)]
Got correct adjacency matrix


Success!

### Task 4.2. Extract a list of triangles from an adjacency matrix


**Inputs:**

   1. The number of vertices in the graph $V (V \leq 6)$.
   
   2. An adjacency matrix describing the graph in the format from task 4.1.
   
**Output:** 

An array of 3-tuples listing all triangles in the graph, that is, all triplets of vertices connected by edges. Each of the 3-tuples should list the triangle vertices in ascending order, and the 3-tuples in the array should be sorted in ascending order as well.

**Example:** 

Consider the adjacency matrix
$$
\begin{bmatrix}
-1 &  0 &  1 \\
 0 & -1 &  2 \\
 1 &  2 & -1
\end{bmatrix}
$$

The list of triangles for it would be `[(0, 1, 2)]`.

In [18]:
%kata T42_AdjacencyMatrixAsTrianglesList

function AdjacencyMatrixAsTrianglesList (V : Int, adjacencyMatrix : Int[][]) : (Int, Int, Int)[] {
    mutable ans = [];
    
    for i in 0..V-1 {
        for k in i+1..V-1 {
            for l in k+1..V-1 {
                if (adjacencyMatrix[i][k] != -1 and 
                    adjacencyMatrix[k][l] != -1 and 
                    adjacencyMatrix[l][i] != -1) {
                        set ans += [(i, k, l)];
                    }
            }
        }
    }
    
    return ans;
}

Running on graph V = 3, edges = []
Got correct triangles list
Running on graph V = 4, edges = [(0, 1),(0, 2),(0, 3),(1, 2),(1, 3),(2, 3)]
Got correct triangles list
Running on graph V = 5, edges = [(4, 0),(2, 1),(3, 1),(3, 2)]
Got correct triangles list
Running on graph V = 5, edges = [(0, 1),(1, 2),(1, 3),(3, 2),(4, 2),(3, 4)]
Got correct triangles list
Running on graph V = 5, edges = [(0, 1),(0, 2),(0, 4),(1, 2),(1, 3),(2, 3),(2, 4),(3, 4)]
Got correct triangles list
Running on graph V = 6, edges = [(0, 1),(0, 2),(0, 4),(0, 5),(1, 2),(1, 3),(1, 5),(2, 3),(2, 4),(3, 4),(3, 5),(4, 5)]
Got correct triangles list


Success!

### Task 4.3. Classical verification of triangle-free coloring
**Inputs:**
   1. The number of vertices in the graph $V (V \leq 6)$.
   
   2. An array of $E$ tuples of integers, representing the edges of the graph $(E \leq 12)$. Each tuple gives the indices of the start and the end vertices of the edge. The vertices are indexed $0$ through $V - 1$.
   
   3. An array of $E$ integers, representing the edge coloring of the graph. $i^{th}$ element of the array is the color of the edge number $i$, and it is $0$ or $1$. The colors of edges in this array are given in the same order as the edges in the "edges" array.

**Output:** 

`true` if the given coloring is triangle-free (i.e., no triangle of edges connecting 3 vertices has all three edges in the same color), and `false` otherwise.

**Example:** 

Consider a graph with $V = 3$ and edges = `[(0, 1), (0, 2), (1, 2)]`. Some of the valid colorings for it would be `[0, 1, 0]` and `[-1, 5, 18]`.

In [25]:
%kata T43_IsVertexColoringTriangleFree

function IsVertexColoringTriangleFree (V : Int, edges: (Int, Int)[], colors: Int[]) : Bool {
    let adjacencyMatrix = EdgesListAsAdjacencyMatrix(V, edges);
    let triangles = AdjacencyMatrixAsTrianglesList(V, adjacencyMatrix);
    
    mutable triangleFree = true;
    
    for triangle in triangles {
        let (v1, v2, v3) = triangle;
        if triangleFree and colors[v1] == colors[v2] and colors[v1] == colors[v3] {
            set triangleFree = false;
        }
    }
    
    return triangleFree;
}

Success!

### Task 4.4. Oracle to check that three colors don't form a triangle (f(x) = 1 if at least two of three input bits are different)

**Inputs:**

  1. a 3-qubit array `inputs`,
  
  2. a qubit `output`.
  
**Goal:** 

Flip the output qubit if and only if at least two of the input qubits are different.

**Example:** 

For the result of applying the operation to state $(|001\rangle + |110\rangle + |111\rangle) \otimes |0\rangle$ will be $|001\rangle \otimes |1\rangle + |110\rangle \otimes |1\rangle + |111\rangle \otimes |0\rangle$.

In [26]:
%kata T44_ValidTriangleOracle

operation ValidTriangleOracle (inputs : Qubit[], output : Qubit) : Unit is Adj+Ctl {
    X(output);
    ControlledOnInt(0, X)(inputs, output);
    ControlledOnInt(7, X)(inputs, output);
}

Success!

### Task 4.5. Oracle for verifying triangle-free edge coloring (f(x) = 1 if the graph edge coloring is triangle-free)

**Inputs:**
  1. The number of vertices in the graph $V (V \leq 6)$.
  
  2. An array of $E$ tuples of integers `edges`, representing the edges of the graph ($0 \leq E \leq V(V-1)/2$). Each tuple gives the indices of the start and the end vertices of the edge. The vertices are indexed $0$ through $V - 1$. The graph is undirected, so the order of the start and the end vertices in the edge doesn't matter.
  
  3. An array of $E$ qubits `colorsRegister` that encodes the color assignments of the edges. Each color will be $0$ or $1$ (stored in 1 qubit). The colors of edges in this array are given in the same order as the edges in the `edges` array.
  
  4. A qubit `target` in an arbitrary state.

**Goal:** 

Implement a marking oracle for function $f(x) = 1$ if the coloring of the edges of the given graph described by this colors assignment is triangle-free, i.e., no triangle of edges connecting 3 vertices has all three edges in the same color.

In this task you are not allowed to use quantum gates that use more qubits than the number of edges in the graph,
unless there are 3 or less edges in the graph. For example, if the graph has 4 edges, you can only use 4-qubit gates or less.
You are guaranteed that in tests that have 4 or more edges in the graph the number of triangles in the graph 
will be strictly less than the number of edges.

**Example:** 
A graph with 3 vertices and 3 edges `[(0, 1), (1, 2), (2, 0)]` has one triangle.  
The result of applying the operation to state $\frac{1}{\sqrt{3}} \big(|001\rangle + |110\rangle + |111\rangle\big) \otimes |0\rangle$ will be $\frac{1}{\sqrt{3}} \big(|001\rangle \otimes |1\rangle + |110\rangle \otimes |1\rangle + |111\rangle \otimes |0\rangle\big)$.
The first two terms describe triangle-free colorings, and the last term describes a coloring where all edges of the triangle have the same color.

<br/>
<details>
  <summary><b>Need a hint? Click here</b></summary>
  Make use of functions and operations you've defined in previous tasks. 
</details>


In [32]:
function triangleVerticesList (V : Int, adjacencyMatrix : Int[][]) : (Int, Int, Int)[] {
    mutable ans = [];
    
    for i in 0..V-1 {
        for k in i+1..V-1 {
            for l in k+1..V-1 {
                if (adjacencyMatrix[i][k] != -1 and 
                    adjacencyMatrix[k][l] != -1 and 
                    adjacencyMatrix[l][i] != -1) {
                        set ans += [(adjacencyMatrix[i][k],
                                     adjacencyMatrix[k][l], 
                                     adjacencyMatrix[l][i])];
                    }
            }
        }
    }
    
    return ans;
}

triangleVerticesList

In [33]:
%kata T45_TriangleFreeColoringOracle

operation TriangleFreeColoringOracle (
    V : Int, 
    edges : (Int, Int)[], 
    colorsRegister : Qubit[], 
    target : Qubit
) : Unit is Adj+Ctl {

    let adjacencyMatrix = EdgesListAsAdjacencyMatrix(V, edges);
    let triangles = triangleVerticesList(V, adjacencyMatrix);
    let N = Length(triangles);
    
    use q = Qubit[N];
    within {
        for i in IndexRange(triangles) {
            let (e1, e2, e3) = triangles[i];
            ValidTriangleOracle([colorsRegister[e1], 
                                 colorsRegister[e2], 
                                 colorsRegister[e3]], q[i]);
        }
    }
    apply {
        ControlledOnInt(2^N-1, X)(q, target);
    }
}

Testing (6, [])
Testing (6, [(0, 1),(1, 2),(2, 3),(3, 4),(4, 5),(5, 0)])
Testing (6, [(0, 1),(0, 2),(0, 3),(0, 4),(0, 5)])
Testing (6, [(0, 1),(0, 3),(0, 5),(1, 2),(1, 4),(2, 3),(2, 5),(3, 4),(4, 5)])
Testing (3, [(0, 1),(1, 2),(2, 0)])
Testing (6, [(0, 1),(4, 3),(2, 1),(5, 4),(5, 0),(2, 0)])
Testing (4, [(1, 0),(3, 2),(0, 3),(2, 1),(3, 1)])
Testing (4, [(1, 0),(3, 2),(0, 3),(2, 1),(3, 1),(0, 2)])
Testing (5, [(0, 2),(1, 2),(3, 2),(4, 2),(0, 1),(1, 3),(4, 0),(3, 4)])
Testing (6, [(2, 1),(2, 3),(1, 3),(1, 0),(1, 5),(3, 5),(3, 4),(0, 5),(5, 4)])


Success!